# Local Experiment Tracking (No External Services)

## Objective
This notebook demonstrates how to track deep learning experiments locally without
using third-party services such as Weights & Biases.

We log:
- Hyperparameters
- Training and validation metrics
- Model artifacts

(W&B unresolved issues)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import models

from torch.utils.data import DataLoader, random_split


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [23]:
config = {
    "batch_size": 64,
    "epochs": 5,
    "learning_rate": 1e-3,
    "num_workers": 2,
    "image_size": 224
}


In [25]:
train_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [37]:
train_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=train_transforms
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=test_transforms
)

train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))


class_names = train_dataset.dataset.classes
print(class_names)


Files already downloaded and verified
Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [38]:
train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=config["num_workers"]
)

test_loader = DataLoader(
    test_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=config["num_workers"]
)


In [39]:
model = models.resnet18(pretrained=True)

c:\Users\lenovo\.conda\envs\mldm2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lenovo\.conda\envs\mldm2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
for param in model.parameters():
    param.requires_grad = False

In [41]:
model.fc = nn.Linear(model.fc.in_features, 10)
model = model.to(device)


In [42]:
criterion = nn.CrossEntropyLoss()

In [43]:
optimizer = optim.Adam(model.fc.parameters(), lr=config["learning_rate"])

In [44]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total


In [45]:
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return running_loss / len(loader), correct / total


In [46]:
for epoch in range(config["epochs"]):
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device
    )

    val_loss, val_acc = validate(
        model, test_loader, criterion, device
    )

    print(
        f"Epoch {epoch+1}/{config['epochs']} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


Epoch 1/5 | Train Acc: 0.5008 | Val Acc: 0.6720
Epoch 2/5 | Train Acc: 0.7242 | Val Acc: 0.7310
Epoch 3/5 | Train Acc: 0.7570 | Val Acc: 0.7450
Epoch 4/5 | Train Acc: 0.7766 | Val Acc: 0.7460
Epoch 5/5 | Train Acc: 0.7870 | Val Acc: 0.7490


In [48]:
torch.save(model.state_dict(), "resnet18_cifar10_transfer.pth")